<a href="https://colab.research.google.com/github/psygrammer/fast_and_slow/blob/master/drl/notebooks/dqn/tf_ch09_Deep_Q_Network_and_Its_Variants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9. Deep Q Network and Its Variants

* Fast And Slow / Deep RL - tf2 [1]
* 김무성

#### 실습 repo

In [1]:
!git clone https://github.com/psygrammer/fast_and_slow

Cloning into 'fast_and_slow'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 77 (delta 20), reused 43 (delta 10), pack-reused 0
Unpacking objects: 100% (77/77), done.


In [2]:
ls

fast_and_slow/  sample_data/


In [3]:
cd /content/fast_and_slow/drl/notebooks/dqn

/content/fast_and_slow/drl/notebooks/dqn


In [4]:
ls

tf_ch09_Deep_Q_Network_and_Its_Variants.ipynb
tf_ch09_Deep_Q_Network_and_Its_Variants_sol.ipynb


#### Install dependancies

In [5]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install pyglet==1.3.2 > /dev/null 2>&1
!apt-get install -y xvfb x11-utils python-opengl ffmpeg > /dev/null 2>&1

#### Imports and Helper functions

In [10]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only

import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [7]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [8]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

-------------------

# Playing Atari games using DQN



In [9]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [ ]:
# Let's implement the DQN to play the Ms Pacman game. 
# First, let's import the necessary libraries:
import random
import gym
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [12]:
# Now, let's create the Ms Pacman game environment using Gym:
env = gym.make("MsPacman-v0")

In [11]:
ls

tf_ch09_Deep_Q_Network_and_Its_Variants.ipynb
tf_ch09_Deep_Q_Network_and_Its_Variants_sol.ipynb


In [13]:
env = wrap_env(env) # monitoring

In [14]:
ls

tf_ch09_Deep_Q_Network_and_Its_Variants.ipynb      video/
tf_ch09_Deep_Q_Network_and_Its_Variants_sol.ipynb


In [15]:
ls video

In [16]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [17]:
# Set the state size:
state_size = (88, 80, 1)

In [18]:
# Get the number of actions:
action_size = env.action_space.n

## Preprocess the game screen

Now, let's define a function called preprocess_state which takes the game state (image of the game screen) as an input and returns the preprocessed game state (image of the game screen):

In [ ]:
color = np.array([210, 164, 74]).mean()
color

149.33333333333334

In [ ]:
def preprocess_state(state):

    #crop and resize the image
    
    #convert the image to greyscale
    
    #improve image contrast
    
    #normalize the image
    
    #reshape the image
    
    return image


## Defining the DQN class

In [ ]:
class DQN:
    # -- Defining the init method
    def __init__(self, state_size, action_size):
        pass
        #define the state size
        
        #define the action size
        
        #define the replay buffer
        
        #define the discount factor
        
        #define the epsilon value
        
        #define the update rate at which we want to update the target network
        
        #define the main network
        
        #define the target network
        
        #copy the weights of the main network to the target network
        

    # -- Building the DQN
    #Let's define a function called build_network which is essentially our DQN. 

    def build_network(self):
        # Define the first convolutional layer:
        
        # Define the second convolutional layer:
        
        # Define the third convolutional layer:
        
        #Flatten the feature maps obtained as a result of the third convolutional layer:
        
        # Feed the flattened maps to the fully connected layer:
        
        # Compile the model with loss as MSE:
        
        # Return the model:
        return model


    # -- Storing the transition
    #We learned that we train DQN by randomly sampling a minibatch of transitions from the
    #replay buffer. So, we define a function called store_transition which stores the transition information
    #into the replay buffer

    def store_transistion(self, state, action, reward, next_state, done):
        pass

    # -- Defining the epsilon-greedy policy
    #We learned that in DQN, to take care of exploration-exploitation trade off, we select action
    #using the epsilon-greedy policy. So, now we define the function called epsilon_greedy
    #for selecting action using the epsilon-greedy policy.
    
    def epsilon_greedy(self, state):
        
        return np.argmax(Q_values[0])


    # -- Define the training
    #train the network
    def train(self, batch_size):
        
        #sample a mini batch of transition from the replay buffer
        
        #compute the Q value using the target network
        for state, action, reward, next_state, done in minibatch:
                
            #compute the Q value using the main network 
            
            #train the main network
            
    #update the target network weights by copying from the main network
    def update_target_network(self):
        pass

## Training the DQN

In [20]:
# Now, let's train the network. 
# First, let's set the number of episodes we want to train the network:


500

In [21]:
# Define the number of time steps


20000

In [22]:
# Define the batch size:


8

In [23]:
# Set the number of past game screens we want to consider:


4

In [ ]:
# Instantiate the DQN class
dqn = DQN(state_size, action_size)

In [ ]:
done = False
time_step = 0

#for each episode
for i in range(num_episodes):
    #set return to 0
    Return = 0
    
    #preprocess the game screen
    
    #for each step in the episode
    for t in range(num_timesteps):
        
        #render the environment
        env.render()
        
        #update the time step
        
        #update the target network
        
        #select the action
        
        #perform the selected action
        
        #preprocess the next state
        
        #store the transition information
        
        #update current state to next state
        
        #update the return
        
        #if the episode is done then print the return
            
        #if the number of transistions in the replay buffer is greater than batch size
        #then train the network
    

In [ ]:
env.close()

In [ ]:
%ls video

In [ ]:
show_video()

--------------------

# 참고문헌
* [1] Deep Reinforcement Learning with Python (2판) - https://www.amazon.com/Deep-Reinforcement-Learning-Python-distributional/dp/1839210680/
